# **Corrector Ortográfico**

<div>
<img src="../assets/spelling.png" width="500" style=" display: block; margin-left: auto; margin-right: auto;"/>
</div>

Vamos a crear una aplicación para revisar la ortografía de nuestra lengua de trabajo. Mientras que Microsoft Word y otros editores de texto admiten corrección ortográfica en lenguas bien documentadas y estudiadas, como el inglés o el español, la mayoría de las lenguas de poco estudiadas no están disponibles en dichas herramientas.

#### ¿Cómo lo hacemos nosotros?
Hay dos enfoques posibles para crear un sistema de correctores ortográficos. 
1. Almacenar una lista enorme de cada palabra en la lengua. Cuando el usuario escriba una palabra, se comprueba si la palabra está en esa lista.
2. Almacenar las *bases/temas* o *raíces* para cada palabra en la lengua. Cuando un usuario escriba una palabra, se comprueba si la palabra es una forma válida de uno de según las bases/temas (almacenadas). Esto requeriría conocimiento sobre la morfología de la lengua.

Aunque pueda parecer lo contrario, la mayoría del software moderno utiliza el **Enfoque 1**. Es más fácil implementar y funciona más rápido.

***

## **Preparación de datos**

### Cargar el corpus

<div>
<img src="../assets/corpus.png" width="500" style=" display: block; margin-left: auto; margin-right: auto;"/>
</div>

El **corpus** que usaremos para este proyecto y futuros proyectos es un corpus [Uspanteko](https://www.ethnologue.com/language/usp) . Uspanteko es una lengua mayense con unos 5.000 hablantes que usa un alfabeto basado en latino y cuya morfología es concatenativa.

Nuestro corpus tiene 23 archivos de texto del Uspanteko. Leeremos todos los archivos y los concatenaremos juntos.

<div class="alert alert-block alert-warning">
    Si quieres usar tu propia lengua y ya tienes un corpus, no dudes en usarlo en su lugar.
</div>

In [ ]:
from typing import List, Dict, Tuple
import os

corpus = ""


# If you're using your own corpus, change this to the correct directory
corpus_directory = "../../corpora/usp"

# Loop over each file in the corpus so we can read it in
for file_name in os.listdir(corpus_directory):
    
    # We will save one corpus entry, 50, for testing
    if file_name == "50.txt":
        continue
        
    
    #  Make sure we only read text files
    if ".txt" not in file_name:
        continue
        
        
    # Read the current file as a string
    file_path = os.path.join(corpus_directory, file_name)
    
    with open(file_path, 'r') as file:
        file_contents = file.read()
        corpus += (file_contents + "\n")
        
print(corpus[:300])

PreprocesamientoVamos a preprocesar un poco nuestro texto para la corrección ortográfica.

In [ ]:
# First, strip accent marks (they aren't usually written in Uspanteko).

from util import strip_accents

corpus = strip_accents(corpus)
strip_accents("ójor taq tziij kita' jaa,")

#### **Ejercicio 1**
Termina la siguiente celda para convertir todo el corpus en minúscula.
<details>
  <summary>Show answer</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">corpus = corpus.lower()</code></pre>
</details>

In [ ]:
# TODO: Finish me!!


print(corpus[:99])

## **Creando una Lista de Palabras**
Vamos a crear una lista de cada palabra que ocurre en nuestro corpus. Ignoraremos los signos de puntuación y asumiremos que una palabra está rodeada de espacios o puntuaciones. Adicionalmente, mantendremos un recuento de la frecuencia de cada palabra para su uso más adelante.

### Tokenización de palabras usando una Expresión Regular (RegEx)

> **Tokenización** se refiere al proceso de romper una cadena de texto en tokens. Los tokens pueden ser palabras, caracteres o morfemas. En este caso, nuestros tokens serán palabras.

Utilizaremos una [expresión regular](./skills/regex.ipynb) que busque grupos de letras y apóstrofes. Cuando ejecutemos la Expresión Regular sobre nuestro texto, cada grupo que esta encuentre será una palabra separada.

Por ejemplo, en la siguiente cadena:

```ójor taq tziij kita' jaa```

La Expresión Regular producirá:

```["ojor", "taq", "tziij", "kita'", "jaa"]```

En Uspanteko, las palabras siempre se dividen por signos de puntuación o por espacios en blanco. Por lo tanto, podemos asumir que cada agrupación que contenga sólo letras debe ser una palabra.

<div class="alert alert-block alert-warning">
Puede que tu lengua necesite de una RegEx personalizada para detectar palabras. Por favor, consulta la lección sobre expresiones regulares para obtener información. También puedes utilizar una herramienta para probar tus RegEx como <a href='https://regex101.com'>regex101</a> y así asegurarse de que su RegEx funciona de la forma que esperas.
</div>

In [ ]:
import re

# If your language uses some other character within words (like hyphens) you may need to update this regex appropriately
word_regex = r"[\w|\']+"

def tokenize(text: str) -> List[str]:
    return re.findall(word_regex, text)

words = tokenize(corpus)
print(words[:100])

### Crear un lexicon

Un **lexicon** se refiere a todo el vocabulario de las palabras utilizadas en el corpus

Para crear un lexicon, iteraremos sobre cada palabra a través de todo el corpus. Utilizamos un [set](./skills/sets.ipynb) para crear una lista de todas las palabras únicas en el lexicon. 

#### **Ejercicio 2**
Crea un conjunto de todas las palabras únicas en tu corpus y almacénalo en una variable llamada `lexicon`. Imprime como resultado el número de elementos en el lexicon.

<details>
  <summary>Show answer</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">lexicon = set()
for word in words:
    lexicon.add(word)</code></pre>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">print(len(lexicon))
</code></pre>
</details>

In [ ]:
# TODO: Loop over the corpus and add each word to the lexicon


# TODO: Print the number of elements in the lexicon


# Store the lexicon to permanent storage so we can retrieve it later if needed
%store lexicon

## **Construyendo un corrector ortográfico**
En este punto, tenemos un léxico con todas nuestras palabras y sus frecuencias. Ahora estamos listos para construir nuestro programa de corrección ortográfica. 

### Detectar palabras mal escritas
Vamos a crear una función que tomará una frase y encontrará cualquier palabra mal hecha. Para cada palabra, la función comprobará si esta está en nuestro lexicon. Si no está en el lexicon, se contará como un error ortográfico.

Ahora mismo, en cualquier momento que encontremos una palabra que no está en nuestro lexicon, la reportaremos como un error ortográfico (incluso si es una nueva palabra escrita correctamente). Mejoraremos esto más tarde.

#### **Ejercicio 3**
Termina el siguiente código para crear una función que encuentra palabras mal hechas.

<details>
  <summary>Show answer</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">def spellcheck(s: str):
    # Preprocess and tokenize
    # TODO: Strip accents
    s = strip_accents(s)<br/>
    # TODO: Make s lowercase
    s = s.lower()<br/>
    # TODO: Tokenize s
    input_tokenized = tokenize(s)<br/>
    # TODO: For each word in the input, check if the word is in the lexicon.
    # If not, add the word to "mispelled_words".
    mispelled_words = []<br/>
    for word in input_tokenized:
        if word not in lexicon:
            mispelled_words.append(word)<br/>
    return mispelled_words
</code></pre>
</details>

<details>
  <summary>Show alternate answer</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">def spellcheck(s: str):
        # Preprocess and tokenize
    # TODO: Strip accents
    s = strip_accents(s)<br/>
    # TODO: Make s lowercase
    s = s.lower()<br/>
    # TODO: Tokenize s
    input_tokenized = tokenize(s)<br/>
    # TODO: For each word in the input, check if the word is in the lexicon.
    # If not, add the word to "mispelled_words".
    mispelled_words = [word for word in input_tokenized if word not in lexicon]
    return mispelled_words</code></pre>
</details>

In [ ]:
def spellcheck(s: str):
    # Preprocess and tokenize
    # TODO: Strip accents
    
    
    # TODO: Make s lowercase
    
    
    # TODO: Tokenize s
    
    
    # TODO: For each word in the input, check if the word is in the lexicon.
    # If not, add the word to "mispelled_words".
    
    # TODO: Return mispelled_words

# This sentence has one mispelling ('tzijj')
test_sentence = "Kwand xink'uli'k', re ójr taq tzijj in ák'el na."
mispelled_words = spellcheck(test_sentence)
print("Mispelled words:", mispelled_words)
print("✅ Correct" if mispelled_words == ['tzijj'] else "❌ Incorrect")

### Encuentra posiciones de palabras mal escritas
En el futuro, tal vez queramos saber *dónde* se encuentran las palabras mal escritas que aparecen en el texto. La siguiente función encuentra la ubicación de palabras erróneas usando nuestra función `spellcheck`.

In [ ]:
def find_mispelled(text: str):
    """Finds mispelled words in a string.
    :return: A list of tuples. Each tuple is (word, index) where `word` is the mispelled word and `index` is the index where it occurs.
    """
    mispelled_words = spellcheck(text)
    
    mispelled_words_and_positions = []
    
    for word in mispelled_words:
        # This regex searches for the given word, surrounded by whitespace or punctuation
        word_regex = f"(^|\W)({word})($|\W)"
        
        # There might be multiple matches if we mispelled a word multiple times
        for match in re.finditer(word_regex, text):
            mispelled_words_and_positions.append({
                'word': word,
                'start': match.start(2),
                'end': match.start(2) + len(word),
                'entity': 'MISPELLED'
            })
        
    # 4. Return the mispelled words, sorted by their position
    return {
        'text': text,
        'entities': sorted(mispelled_words_and_positions, key = lambda x: x['start'])
    }

print(find_mispelled(test_sentence))

## **Haz que el corrector ortográfico sea una aplicación independiente**

Nuestra función `find_mispelled` trabaja para detectar errores ortográficos. Pero esto no es una gran herramienta para que un usuario pueda usarlo. Vamos a crear una aplicación independiente que los usuarios puedan usar.

Utilizaremos [Gradio](https://gradio.app), un framework gratuito que te permite convertir el código de Python en aplicaciones web compartibles. Usar Gradio es tan fácil como tres líneas de código.

In [ ]:
import gradio as gr

gr.close_all()

spellchecker = gr.Interface(fn=find_mispelled, inputs="text", outputs="text", live=True)
spellchecker.launch(share=True)

### Mejorando la interfaz de usuario

Esto funciona muy bien, e incluso podemos compartir nuestra aplicación usando el enlace web de arriba. Ahora, hagamos de la interfaz un poco más agradable.

In [ ]:
gr.close_all()

with gr.Blocks(theme=gr.themes.Soft(), title="Uspanteko Spellchecker") as spellchecker:
    gr.Markdown("# Uspanteko Spellchecker")
    
    with gr.Row():
        input_textbox = gr.Textbox(label="Text", info="Text to spellcheck", lines=3)
        output_textbox = gr.HighlightedText(label="Spellchecked", combine_adjacent=True)
    
    gr.Examples(
        examples=["Kwand xink'uli'k', re ójr taq tzijj in ák'el na."],
        inputs=input_textbox,
        outputs=output_textbox,
        fn=find_mispelled,
    )
    
    # Connect the input to the output using our function
    input_textbox.change(find_mispelled, input_textbox, output_textbox)
    
    
spellchecker.launch(share=True)

## **Permitiendo nuevas palabras**
Hay algunos problemas con nuestra aplicación. Primero, si el usuario escribe una palabra que está escrita correctamente, pero no aparece en nuestro corpus, esta será marcada como incorrecta.

¿Recuerdas cuando guardamos un archivo de nuestro corpus para la prueba? Veamos cuántas palabras no vistas ocurren en ese archivo.

In [ ]:
test_text = ""

with open("../../corpora/usp/50.txt", 'r') as file:
    test_text = file.read()

len(find_mispelled(test_text)['entities'])

### Implementando el botón "Añadir Palabra" en la interfaz de usuario

Hay un montón de errores ortográficos falsos detectados! Esto debido a que nuestro sistema fue construido utilizando sólo un pequeño corpus y por lo tanto no contendrá todas las palabras disponibles (y correctas) de la lengua. Las herramientas comunes de procesamiento de textos resuelven este problema permitiendo al usuario añadir una palabra al lexicon, así que modificaremos nuestra herramienta para hacerlo. 

Para hacer esto, introduciremos la idea de [State de Gradio](https://gradio.app/docs/#state). Mantener un `State` nos permite usar una variable global en nuestras interfaces de Gradio. Utilizaremos una variable `State` para hacer un seguimiento de las palabras equivocadas.

#### **Ejercicio 4**
Termina la función `add_word_to_lexicon` para añadir la `palabra` al `lexicon`. El argumento `mispelled` es el resultado de llamar a `find_mispelled`.

<details>
  <summary>Show answer</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">def add_word_to_lexicon(mispelled):
    first_mispelled_word = mispelled['entities'][0]['word']<br/>
    lexicon.add(first_mispelled_word)</code></pre>
</details>

In [ ]:
gr.close_all()

def add_word_to_lexicon(mispelled):
    # TODO: Add the first mispelled word to the lexicon
    

# NEW: We'll use this function when the input changes instead of `find_mispelled`. 
# We need to update 1) the output text, 2) the button, and 3) the global state
def input_text_changed(text):
    mispelled = find_mispelled(text)
    
    should_show_button = len(mispelled['entities']) > 0
    first_mispelled_word = mispelled['entities'][0]['word'] if should_show_button else ''

    # Here we use an "update" function to change the textbox properties. Learn more: https://gradio.app/docs/#update
    add_to_lexicon_button_updater = gr.Button.update(visible=should_show_button, 
                                                     value=f"Add '{first_mispelled_word}' to lexicon")

    return mispelled, add_to_lexicon_button_updater, mispelled

    
# Interface
with gr.Blocks(theme=gr.themes.Soft(), title="Uspanteko Spellchecker") as spellchecker:
    # NEW: A 'State' variable that keeps track of the mispelled words
    mispelled_words_state = gr.State(None)
    
    gr.Markdown("# Uspanteko Spellchecker")
    
    with gr.Row():
        input_textbox = gr.Textbox(label="Text", info="Text to spellcheck", lines=3)
        
        with gr.Column():
            output_textbox = gr.HighlightedText(label="Spellchecked", combine_adjacent=True)
            
            # NEW: A button that adds the first mispelled word to the lexicon
            add_word_button = gr.Button(value=f"Add word to lexicon", visible=False)
            
    gr.Examples(
        examples=["Kwand xink'uli'k', re ójr taq tzijj in ák'el na."],
        inputs=input_textbox,
        outputs=output_textbox,
        fn=find_mispelled,
    )
    
    input_textbox.change(input_text_changed, input_textbox, [output_textbox, add_word_button, mispelled_words_state])
    
    # NEW: Run a function when the button is called, then update the state
    add_word_button \
        .click(add_word_to_lexicon, [mispelled_words_state]) \
        .then(input_text_changed, input_textbox, [output_textbox, add_word_button, mispelled_words_state])
    
        
spellchecker.launch(share=True)

Ahora, podemos añadir fácilmente palabras que estén correctamente escritas a nuestro diccionario, y no se marcarán como errores en el futuro.

## **Corrección Ortográfica**
Por último, sería bueno actualizar nuestro corrector ortográfico para que dé sugerencias para la correcta ortografía cuando haya habido un error. Para ello, necesitamos determinar qué palabra en nuestro léxico está más cerca de lo que se ha escrito. Utilizaremos la **distancia de edición**, una medida de cuántas ediciones (adiciones, eliminaciones, cambios) se necesitan para pasar de una cadena a otra.

In [ ]:
import nltk

def spelling_suggestions(mispelled_word, n):
    # 1. Calculate the edit distance between the word and every word in the lexicon
    candidate_spellings = []
    
    for word in lexicon:
        edit_distance = nltk.edit_distance(word, mispelled_word)
        candidate_spellings.append((word, edit_distance))
    
    # 2. Find the top n closest words
    sorted_candidates = sorted(candidate_spellings, key=lambda x: (x[1]))
    top_n_candidates = sorted_candidates[:n]
    top_n_words_only = [candidate[0] for candidate in top_n_candidates]
    return top_n_words_only

spelling_suggestions("tzijj", 3)

### Implementando sugerencias en la interfaz de usuario
¡Vamos a implementar esta funcionalidad en la interfaz de usuario!

In [ ]:
gr.close_all()

def input_text_changed2(text):
    mispelled = find_mispelled(text)
    
    should_show_button = len(mispelled['entities']) > 0
    first_mispelled_word = mispelled['entities'][0]['word'] if should_show_button else ''

    add_to_lexicon_button_updater = gr.Button.update(visible=should_show_button, 
                                                     value=f"Add '{first_mispelled_word}' to lexicon")
    
    # NEW: Generate suggestions for the closest spelling
    suggestions = spelling_suggestions(first_mispelled_word, 3)
    suggestions_text = " | ".join(suggestions)

    return mispelled, add_to_lexicon_button_updater, mispelled, suggestions_text

    
# Interface
with gr.Blocks(theme=gr.themes.Soft(), title="Uspanteko Spellchecker") as spellchecker:
    mispelled_words_state = gr.State(None)
    
    gr.Markdown("# Uspanteko Spellchecker")
    
    with gr.Row():
        input_textbox = gr.Textbox(label="Text", info="Text to spellcheck", lines=3)
        
        with gr.Column():
            output_textbox = gr.HighlightedText(label="Spellchecked", combine_adjacent=True)
            add_word_button = gr.Button(value=f"Add word to lexicon", visible=False)
            
            # NEW: Show suggested spellings
            suggestions_textbox = gr.Textbox(label="Suggestions", interactive=False)
            
    gr.Examples(
        examples=["Kwand xink'uli'k', re ójr taq tzijj in ák'el na."],
        inputs=input_textbox,
        outputs=output_textbox,
        fn=find_mispelled,
    )
    
    input_textbox.change(input_text_changed2, input_textbox, [output_textbox, add_word_button, mispelled_words_state, suggestions_textbox])
    
    add_word_button \
        .click(add_word_to_lexicon, [mispelled_words_state]) \
        .then(input_text_changed2, input_textbox, [output_textbox, add_word_button, mispelled_words_state, suggestions_textbox])
    
        
spellchecker.launch(share=True)

**Resumen**
En este tutorial, hemos creado un corrector ortográfico para una lengua poco estudiada/documentada. Esto incluye:
- Construir un lexicon a partir de textos
- Detectar palabras mal escritas
- Predecir la ortografía correcta usando métricas de similitud

### **Desafíos**
1. Ahora mismo, nuestro corrector ortográfico sólo te permite añadir la primera palabra errónea al lexicon. Mejora esta funcionalidad creando dos botones nuevos. Un botón que te permita **Agregar todas** palabras erróneas al léxico. El otro botón que te permita **Ignorar** una palabra equivocada y moverte a la siguiente. 
2. Nuestro corrector ortográfico muestra sugerencias, pero no te permite hacer nada con ellas. Reemplaza el cuadro de texto con botones para cada sugerencia. Cuando hagas clic en una sugerencia, esto debería reemplazar la palabra errónea en el cuadro de texto.